### 特征匹配

####  Brute-Force蛮力匹配

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
img1 = cv2.imread('box.png', 0)
img2 = cv2.imread('box_in_scene.png', 0)

In [3]:
def cv_show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
cv_show('img1',img1)

In [5]:
cv_show('img2',img2)

In [6]:
sift = cv2.xfeatures2d.SIFT_create()

In [7]:
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

In [8]:
# crossCheck表示两个特征点要互相匹，
# 例如A中的第i个特征点与B中的第j个特征点最近的，
# 并且B中的第j个特征点到A中的第i个特征点也是最近的
# 默认使用Norm_l2
#NORM_L2: 归一化数组的(欧几里德距离)，如果其他特征计算方法需要考虑不同的匹配计算方式
# BFMatcher就是：Brute-Force蛮力匹配
bf = cv2.BFMatcher(crossCheck=True)

#### 1对1的匹配
特征点一对一的匹配：一个特征点对应一个特征点

In [15]:
# 对两类特征向量进行匹配
matches = bf.match(des1, des2)
# 并且排序
matches = sorted(matches, key=lambda x: x.distance)

print(type(matches))
print(len(matches))

<class 'list'>
392


In [36]:
help(bf.match)

Help on built-in function match:

match(...) method of cv2.BFMatcher instance
    match(queryDescriptors, trainDescriptors[, mask]) -> matches
    .   @brief Finds the best match for each descriptor from a query set.
    .   
    .   @param queryDescriptors Query set of descriptors.
    .   @param trainDescriptors Train set of descriptors. This set is not added to the train descriptors
    .   collection stored in the class object.
    .   @param matches Matches. If a query descriptor is masked out in mask , no match is added for this
    .   descriptor. So, matches size may be smaller than the query descriptors count.
    .   @param mask Mask specifying permissible matches between an input query and train matrices of
    .   descriptors.
    .   
    .   In the first variant of this method, the train descriptors are passed as an input argument. In the
    .   second variant of the method, train descriptors collection that was set by DescriptorMatcher::add is
    .   used. Optional mas

In [43]:
help(cv2.findHomography)

Help on built-in function findHomography:

findHomography(...)
    findHomography(srcPoints, dstPoints[, method[, ransacReprojThreshold[, mask[, maxIters[, confidence]]]]]) -> retval, mask
    .   @brief Finds a perspective transformation between two planes.
    .   
    .   @param srcPoints Coordinates of the points in the original plane, a matrix of the type CV_32FC2
    .   or vector\<Point2f\> .
    .   @param dstPoints Coordinates of the points in the target plane, a matrix of the type CV_32FC2 or
    .   a vector\<Point2f\> .
    .   @param method Method used to compute a homography matrix. The following methods are possible:
    .   -   **0** - a regular method using all the points, i.e., the least squares method
    .   -   **RANSAC** - RANSAC-based robust method
    .   -   **LMEDS** - Least-Median robust method
    .   -   **RHO** - PROSAC-based robust method
    .   @param ransacReprojThreshold Maximum allowed reprojection error to treat a point pair as an inlier
    .   (us

In [18]:
# 再进行显示
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None,flags=2)

In [19]:
cv_show('img3',img3)

#### k对最佳匹配

In [42]:
bf = cv2.BFMatcher()
# 一个点对应k个点也可
# 为每个关键点绘制两条匹配线
matches = bf.knnMatch(des1, des2, k=2)

print(type(matches))
print(len(matches))
print(len(matches[0]))
print(matches[1][0].trainIdx)
print(matches[1][0].queryIdx)
print(matches[6][0].imgIdx)

<class 'list'>
603
2
336
1
0


In [25]:
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])
print(type(good))

<class 'list'>


In [14]:
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)

In [15]:
cv_show('img3',img3)

如果需要更快速完成操作，可以尝试使用cv2.FlannBasedMatcher

### 随机抽样一致算法（Random sample consensus，RANSAC）

![title](ransac_1.png)

选择初始样本点进行拟合，给定一个容忍范围，不断进行迭代

![title](ransac_2.png)

每一次拟合后，容差范围内都有对应的数据点数，找出数据点个数最多的情况，就是最终的拟合结果

![title](ransac_3.png)

#### 单应性矩阵

![title](ransac_4.png)